In [129]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [130]:
import pickle
from pprint import pprint
from RecData import RecData
from SVD import FastLogisticSVD
from scipy.sparse import lil_array
from sklearn.metrics.pairwise import cosine_similarity
import time

In [132]:
with open('saved_models/2023-6-29-11-36-31_0-3559.pkl', 'rb') as file:
    train_data, svd = pickle.load(file)

In [ ]:
j = 0
# Test top-n list for user j
top_n = svd.top_n(j, 20)
print("Predicted top n:")
pprint([(r, train_data.index_to_title(index)) for r, index in top_n])

# User j true top n list
print("True top n:")
true_top = train_data.top_n(j, 5000)
pprint([train_data.index_to_title(index) for _, index in true_top])

In [25]:
q = svd._item_features
distances = lil_array((q.shape[0], q.shape[0]))

In [133]:
svd.compute_sims()

AttributeError: 'FastLogisticSVD' object has no attribute 'compute_sims'

In [37]:
start_t = time.time()
print("Computing similarities...")
for i in range(q.shape[0]):
    if i % 200 == 0:
        print("Upto row", i)
    for j in range(i, q.shape[0]):
        distances[i, j] = cosine_similarity(q[[i], :], q[[j], :])
print("Done computing similarities in", time.time() - start_t, "seconds")

Computing similarities...
Upto row 0
Upto row 200
Upto row 400
Upto row 600
Upto row 800
Upto row 1000
Upto row 1200
Upto row 1400
Upto row 1600
Upto row 1800
Upto row 2000
Upto row 2200
Done computing similarities in 397.02253437042236 seconds


In [87]:
def knn(sims, i, n=10):
    top = []
    for j in range(sims.shape[0]):
        if j == i:
            continue     
        elif j < i:
            sim = sims[j, i]
        elif j > i:
            sim = sims[i, j]
        
        top.append((sim, j))
        top.sort(reverse=True)
        top = top[:n]
    return top

def many_knn(sims, subjects, n=10):
    top = []
    for j in range(sims.shape[0]):
        total = 0
        for i, pref in subjects:
            if j == i:
                continue     
            elif j < i:
                sim = sims[j, i]
            elif j > i:
                sim = sims[i, j]
                
            if pref == 0:
                sim = 1 - sim
            
            total += sim
        
        avg = total/len(subjects)
        top.append((avg, j))
        top.sort(reverse=True)
        top = top[:n]
    return top
    

In [120]:
train_data.search_title('garry')

[("Garry's Mod", 518)]

In [82]:
subject = 286
print(train_data.index_to_title(subject))
top = knn(distances, subject)

Forza Horizon 5


In [83]:
pprint([(sim, train_data.index_to_title(index)) for sim, index in top])

[(0.9394804654801193, 'First Class Trouble'),
 (0.9390738257725284, 'Construction Simulator'),
 (0.9388757138886102, 'ONE PIECE World Seeker'),
 (0.9387790677492647, 'Injustice™ 2'),
 (0.9381126565121917, 'CarX Drift Racing Online'),
 (0.9379971983512589, 'Sniper Elite 5'),
 (0.9376536716630474, 'Dreamfall Chapters'),
 (0.937325927337768, 'GROUND BRANCH'),
 (0.937098638443306, 'We Were Here Forever'),
 (0.9369496971470657, 'WorldBox - God Simulator')]


In [96]:
top = many_knn(distances, [(59, 1), (342, 1), (500, 1), (1045, 0), (369, 1)])
pprint([(sim, train_data.index_to_title(index)) for sim, index in top])

[(0.7588686574088879, 'Into the Radius VR'),
 (0.7586831056619872, 'Incredibox'),
 (0.7574628412318944, 'Raw Data'),
 (0.7566262426901261, 'Unrailed!'),
 (0.7563903594939038, 'Ancient Dungeon'),
 (0.7557003869512007, 'VTOL VR'),
 (0.7556890685926823, 'Stellaris'),
 (0.755667119731426, 'SpaceEngine'),
 (0.7556102022104467, 'ShellShock Live'),
 (0.7554897018783995, 'Ultimate Chicken Horse')]


In [121]:
top = many_knn(distances, [(518, 1)])
pprint([(sim, train_data.index_to_title(index)) for sim, index in top])

[(0.9573679391179264, 'Portal 2'),
 (0.9553338852238082, 'Counter-Strike: Source'),
 (0.9519024146599377, 'Left 4 Dead'),
 (0.9510782352816002, 'Terraria'),
 (0.9492291504247882, 'POSTAL 2'),
 (0.9487868317567478, 'Left 4 Dead 2'),
 (0.948460726972419, 'Soundpad'),
 (0.94801523483025, 'Bloons TD 6'),
 (0.9476275792463073, 'Ninja Kiwi Archive'),
 (0.9474181740980063, 'Day of Defeat')]
